In [46]:
from jupyter_plotly_dash import JupyterDash

import base64

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))

#########################
# Dashboard Layout / View
#########################                            
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            html.Center(html.B(html.H1('Grazioso Salvare Animal Rescue by Luke Lauck'))),
        ]
    ),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Hr(),
    html.Div([
         dcc.RadioItems(
             id='filter-type',
             options=[
                 {'label': 'Water Rescue', 'value': 'Water'}, 
                 {'label': 'Mountain Rescue', 'value': 'Mountain'},
                 {'label': 'Disaster Rescue', 'value': 'Disaster'},
                 {'label': 'Reset', 'value': 'Reset'}],
             value='Reset',
             labelStyle={"display": "inline-block"},
         )
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(dcc.Graph(id='graph-id')),
            html.Div(
                id='map-id',
                className='col s12 m6')])
])

#############################################
# Interaction Between Components / Controller
#############################################

rescue_type = {
    'Water': {
        'breed': [
            'Labrador Retriever Mix',
            'Chesapeake Bay Retriever',
            'Newfoundland'],
        'sex': ['Intact Female'],
        'age': [26, 156]
    },
    'Mountain': {
        'breed': [
            'German Shepherd',
            'Alaskan',
            'Malamute', 
            'Old English',
            'Sheepdog', 
            'Siberian Husky',
            'Rottweiler'],
        'sex': ['Intact Male'],
        'age': [26, 156]
    },
    'Disaster': {
        'breed': [
            'Doberman Pinscher', 
            'German Shepherd', 
            'Golden Retriever',
            'Bloodhound', 
            'Rottweiler'
        ],
        'sex': ['Intact Male'],
        'age': [20, 300]
    }
}
def getFilteredRescueType(filter_type):
    if filter_type == "Reset":
        return df.to_dict('records')
    else:
        return (df[df['breed'].isin(rescue_type[filter_type]['breed']) &
             df['sex_upon_outcome'].isin(rescue_type[filter_type]['sex']) &
             (df['age_upon_outcome_in_weeks'] > rescue_type[filter_type]['age'][0]) &
             (df['age_upon_outcome_in_weeks'] < rescue_type[filter_type]['age'][1])]).to_dict('records')

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','page_current')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    return (getFilteredRescueType(filter_type), 0)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "figure"),
    [Input('filter-type', 'value')])
def update_graphs(filter_type):
    data = getFilteredRescueType(filter_type);
    filtered_values = {}
    for animal in data:
        if animal['breed'] in filtered_values:
            filtered_values[animal['breed']] += 1
        else:
            filtered_values[animal['breed']] = 1
    names = []
    values = []
    other = 0
    for k,v in filtered_values.items():
        if (v/len(filtered_values) < .1):
            other += v/len(filtered_values)
            continue
        names.append(k)
        values.append(v)
    if other > 0:
        names.append('Others')
        values.append(other)
    return go.Figure(data=[go.Pie(labels=names, values=values)])

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'data'),
     Input('datatable-id', "selected_rows")])
def update_map(data, row):
    animal = data[row[0]]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[animal['location_lat'],animal['location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[animal['location_lat'],animal['location_long']], children=[
                dl.Tooltip(animal['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal['name'])
                ])
            ])
        ])
    ]

app